# Synthetic Optical Flow from Fused Lidar


In [1]:
import sys
sys.path.append('/opt/psegs')

import numpy as np

from psegs.exp.fused_lidar_flow import FusedLidarCloudTableBase
from psegs.exp.fused_lidar_flow import TaskLidarCuboidCameraDFFactory
from psegs.exp.fused_lidar_flow import OpticalFlowRenderBase

import IPython.display
import PIL.Image


## General Notebook Utilities
    
def imshow(x):
    IPython.display.display(PIL.Image.fromarray(x))

def show_html(x):
    from IPython.core.display import display, HTML
    display(HTML(x))

## SemanticKITTI

In [2]:
from psegs.exp.semantic_kitti import SemanticKITTISDTable

class SemanticKITTILCCDFFactory(TaskLidarCuboidCameraDFFactory):
    
    SRC_SD_TABLE = SemanticKITTISDTable
    
    @classmethod
    def build_df_for_segment(cls, spark, segment_uri):
        seg_rdd = cls.SRC_SD_TABLE.get_segment_datum_rdd(spark, segment_uri)
        
        def to_task_row(scan_id_iter_sds):
            scan_id, iter_sds = scan_id_iter_sds
            camera_images = []
            point_clouds = []
            for sd in iter_sds:
                if sd.camera_image is not None:
                    camera_images.append(sd)
                elif sd.point_cloud is not None:
                    point_clouds.append(sd)
            
            from pyspark import Row
            r = Row(
                    task_id=int(scan_id),
                    pc_sds=point_clouds,
                    cuboids_sds=[], # SemanticKITTI has no cuboids
                    ci_sds=camera_images) 
            from oarphpy.spark import RowAdapter
            return RowAdapter.to_row(r)
            
        grouped = seg_rdd.groupBy(lambda sd: sd.uri.extra['semantic_kitti.scan_id'])
        row_rdd = grouped.map(to_task_row)

        df = spark.createDataFrame(row_rdd, schema=cls.table_schema())
        df = df.persist()
        return df

class SemanticKITTIFusedWorldCloudTable(FusedLidarCloudTableBase):
    TASK_DF_FACTORY = SemanticKITTILCCDFFactory

    # SemanticKITTI has no cuboids, so we skip this step.
    HAS_OBJ_CLOUDS = False

class SemanticKITTIOFlowRenderer(OpticalFlowRenderBase):
    FUSED_LIDAR_SD_TABLE = SemanticKITTIFusedWorldCloudTable

## KITTI-360

In [3]:
from psegs.datasets.kitti_360 import KITTI360SDTable
class KITTI360OurFusedClouds(KITTI360SDTable):
    INCLUDE_FISHEYES = False
    INCLUDE_FUSED_CLOUDS = False  # Use our own fused clouds

class KITTI360LCCDFFactory(TaskLidarCuboidCameraDFFactory):
    
    SRC_SD_TABLE = KITTI360OurFusedClouds
    
    @classmethod
    def build_df_for_segment(cls, spark, segment_uri):
        datum_df = cls.SRC_SD_TABLE.get_segment_datum_df(spark, segment_uri)
        datum_df.registerTempTable('datums')
        print('Building tasks table for %s ...' % segment_uri.segment_id)
        
        spark.catalog.dropTempView('kitti360_tasks_df')
        task_data_df = spark.sql("""
            CACHE TABLE kitti360_tasks_df OPTIONS ( 'storageLevel' 'DISK_ONLY' ) AS
            SELECT 
              INT(uri.extra.`kitti-360.frame_id`) AS task_id,
              COLLECT_LIST(STRUCT(__pyclass__, uri, point_cloud)) 
                  FILTER (WHERE uri.topic LIKE '%lidar%') AS pc_sds,
              COLLECT_LIST(STRUCT(__pyclass__, uri, cuboids)) 
                  FILTER (WHERE uri.topic LIKE '%cuboid%') AS cuboids_sds,
              COLLECT_LIST(STRUCT(__pyclass__, uri, camera_image)) 
                  FILTER (WHERE uri.topic LIKE '%camera%') AS ci_sds
            FROM datums
            WHERE (
              uri.topic LIKE '%cuboid%' OR
              uri.topic LIKE '%lidar%' OR
              uri.topic LIKE '%camera%'
            ) AND (
              camera_image is NULL OR (camera_image.extra.`kitti-360.has-valid-ego-pose` = 'True')
            ) AND (
              point_cloud is NULL OR (point_cloud.extra.`kitti-360.has-valid-ego-pose` = 'True')
            )
            GROUP BY task_id
        """)
        
        tasks_df = spark.sql('SELECT * FROM kitti360_tasks_df')
        print('... done.')
        return tasks_df
        
    
class KITTI360WorldCloudTableBase(FusedLidarCloudTableBase):
    TASK_DF_FACTORY = KITTI360LCCDFFactory
        
class KITTI360OFlowRenderer(OpticalFlowRenderBase):
    FUSED_LIDAR_SD_TABLE = KITTI360WorldCloudTableBase

    
    
# class KITTI360OurFusedWorldCloudTable(FusedLidarCloudTableBase):
#     SRC_SD_TABLE = KITTI360OurFusedClouds
    
#     @classmethod
#     def _get_task_lidar_cuboid_rdd(cls, spark, segment_uri):
#         datum_df = cls.SRC_SD_TABLE.get_segment_datum_df(spark, segment_uri)
#         datum_df.registerTempTable('datums')
#         spark.catalog.dropTempView('culi_tasks_df')
#         print('Building tasks table for %s ...' % segment_uri.segment_id)
#         spark.sql("""
#           CACHE TABLE culi_tasks_df OPTIONS ( 'storageLevel' 'DISK_ONLY' ) AS
#           SELECT 
#               CONCAT(uri.segment_id, '.', uri.extra.`kitti-360.frame_id`) AS task_id,
#               FLATTEN(COLLECT_LIST(cuboids)) AS cuboids, 
#               COLLECT_LIST(point_cloud) AS point_clouds
#           FROM datums
#           WHERE 
#               uri.topic LIKE '%cuboid%' OR uri.topic LIKE '%lidar%'
#           GROUP BY task_id
#         """)
        
        
#         # TODO! for lidar and camera image!
#         #         both_have_ego_pose = (
#         #             ci1.extra.get('kitti-360.has-valid-ego-pose') and
#         #             ci2.extra.get('kitti-360.has-valid-ego-pose'))
        
#         tasks_df = spark.sql('SELECT * FROM culi_tasks_df')
#         print('... done.')
#         return tasks_df.rdd


## NuScenes

In [4]:
# !pip3 install nuscenes-devkit==1.1.2
from psegs.datasets.nuscenes import NuscStampedDatumTableBase
from psegs.datasets.nuscenes import NuscStampedDatumTableLabelsAllFrames


class NuscKFOnlyLCCDFFactory(TaskLidarCuboidCameraDFFactory):
    
    SRC_SD_TABLE = NuscStampedDatumTableBase
    
    @classmethod
    def build_df_for_segment(cls, spark, segment_uri):
        datum_df = cls.SRC_SD_TABLE.get_segment_datum_df(spark, segment_uri)
        datum_df.registerTempTable('datums')
        print('Building tasks table for %s ...' % segment_uri.segment_id)
        
        # Nusc doesn't have numerical task_ids so we'll have to induce
        # one via lidar timestamp.
        # NB: for Nusc: can group by nuscenes-sample-token FOR KEYFRAMES-ONLY DATA
        task_data_df = spark.sql("""
            SELECT 
              COLLECT_LIST(STRUCT(__pyclass__, uri, point_cloud)) 
                  FILTER (WHERE uri.topic LIKE '%lidar%') AS pc_sds,
              COLLECT_LIST(STRUCT(__pyclass__, uri, cuboids)) 
                  FILTER (WHERE uri.topic LIKE '%cuboid%') AS cuboids_sds,
              COLLECT_LIST(STRUCT(__pyclass__, uri, camera_image)) 
                  FILTER (WHERE uri.topic LIKE '%camera%') AS ci_sds,
              MIN(uri.timestamp) FILTER (WHERE uri.topic LIKE '%lidar%') AS lidar_time,
              FIRST(uri.extra.`nuscenes-sample-token`) AS sample_token
            FROM datums
            WHERE 
            uri.extra.`nuscenes-is-keyframe` = 'True' AND (
              uri.extra['nuscenes-label-channel'] is NULL OR 
              uri.extra['nuscenes-label-channel'] LIKE '%LIDAR%'
            ) AND (
              uri.topic LIKE '%cuboid%' OR
              uri.topic LIKE '%lidar%' OR
              uri.topic LIKE '%camera%'
            )
            GROUP BY uri.extra.`nuscenes-sample-token`
            ORDER BY lidar_time
        """)
        sample_tokens_ordered = [r.sample_token for r in task_data_df.select('sample_token').collect()]
        task_to_stoken = [
            {'task_id': task_id, 'sample_token': sample_token}
            for task_id, sample_token in enumerate(sample_tokens_ordered)
        ]
        task_id_rdd = spark.sparkContext.parallelize(task_to_stoken)
        task_id_df = spark.createDataFrame(task_id_rdd)
        tasks_df = task_data_df.join(task_id_df, on=['sample_token'], how='inner')
        tasks_df = tasks_df.persist()
        print('... done.')
        return tasks_df


class NuscAllFramesLCCDFFactory(TaskLidarCuboidCameraDFFactory):
    
    SRC_SD_TABLE = NuscStampedDatumTableLabelsAllFrames
    
    @classmethod
    def build_df_for_segment(cls, spark, segment_uri):
        datum_df = cls.SRC_SD_TABLE.get_segment_datum_df(spark, segment_uri)
        datum_df.registerTempTable('datums')
        print('Building tasks table for %s ...' % segment_uri.segment_id)
        
        task_data_df = spark.sql("""
            SELECT 
              COLLECT_LIST(STRUCT(__pyclass__, uri, point_cloud)) 
                  FILTER (WHERE uri.topic LIKE '%lidar%') AS pc_sds,
              COLLECT_LIST(STRUCT(__pyclass__, uri, cuboids)) 
                  FILTER (WHERE uri.topic LIKE '%cuboid%') AS cuboids_sds,
              COLLECT_LIST(STRUCT(__pyclass__, uri, camera_image)) 
                  FILTER (WHERE uri.topic LIKE '%camera%') AS ci_sds,
              MIN(uri.timestamp) FILTER (WHERE uri.topic LIKE '%lidar%') AS lidar_time,
              FIRST(uri.extra.`nuscenes-sample-token`) AS sample_token
            FROM datums
            WHERE 
            (
              uri.extra['nuscenes-label-channel'] is NULL OR 
              uri.extra['nuscenes-label-channel'] LIKE '%LIDAR%'
            ) AND (
              uri.topic LIKE '%cuboid%' OR
              uri.topic LIKE '%lidar%' OR
              uri.topic LIKE '%camera%'
            )
            GROUP BY uri.extra.`nuscenes-sample-token`
            ORDER BY lidar_time
        """)
        sample_tokens_ordered = [r.sample_token for r in task_data_df.select('sample_token').collect()]
        task_to_stoken = [
            {'task_id': task_id, 'sample_token': sample_token}
            for task_id, sample_token in enumerate(sample_tokens_ordered)
        ]
        task_id_rdd = spark.sparkContext.parallelize(task_to_stoken)
        task_id_df = spark.createDataFrame(task_id_rdd)
        tasks_df = task_data_df.join(task_id_df, on=['sample_token'], how='inner')
        tasks_df = tasks_df.persist()
        print('... done.')
        return tasks_df
        
        
        
        
        
        
        
        
        
        
        
        
#         # Nusc doesn't have numerical task_ids so we'll have to induce
#         # one via lidar timestamp.
#         task_data_df = spark.sql("""
#             SELECT 
#               COLLECT_LIST(STRUCT(__pyclass__, uri, point_cloud)) 
#                   FILTER (WHERE uri.topic LIKE '%lidar%') AS pc_sds,
#               COLLECT_LIST(STRUCT(__pyclass__, uri, cuboids)) 
#                   FILTER (WHERE uri.topic LIKE '%cuboid%') AS cuboids_sds,
#               COLLECT_LIST(STRUCT(__pyclass__, uri, camera_image)) 
#                   FILTER (WHERE uri.topic LIKE '%camera%') AS ci_sds,
#               MIN(uri.timestamp) FILTER (WHERE uri.topic LIKE '%lidar%') AS lidar_time,
#               FIRST(uri.extra.`nuscenes-sample-token`) AS sample_token
#             FROM datums
#             WHERE 
#              (
#                uri.extra['nuscenes-label-channel'] is NULL OR 
#                uri.extra['nuscenes-label-channel'] LIKE '%LIDAR%'
#              ) AND (
#                uri.topic LIKE '%cuboid%' OR
#                uri.topic LIKE '%lidar%' OR
#                uri.topic LIKE '%camera%'
#              )
            
#            SELECT 
#                CONCAT(uri.segment_id, '.', uri.timestamp) AS task_id,
#                FLATTEN(COLLECT_LIST(cuboids)) AS cuboids, 
#                COLLECT_LIST(point_cloud) AS point_clouds
#            FROM datums
           
#            WHERE 
#              (
#                uri.extra['nuscenes-label-channel'] is NULL OR 
#                uri.extra['nuscenes-label-channel'] LIKE '%LIDAR%'
#              ) AND (
#                uri.topic LIKE '%cuboid%' OR
#                uri.topic LIKE '%lidar%'
#              )
#            GROUP BY task_id
#            HAVING SIZE(cuboids) > 0 AND SIZE(point_clouds) > 0
#          """)
#         # #             spark.sql("""
#         # #               CACHE TABLE culi_tasks_df OPTIONS ( 'storageLevel' 'DISK_ONLY' ) AS
#         # #               SELECT 
#         # #                   CONCAT(uri.segment_id, '.', uri.timestamp) AS task_id,
#         # #                   FLATTEN(COLLECT_LIST(cuboids)) AS cuboids, 
#         # #                   COLLECT_LIST(point_cloud) AS point_clouds
#         # #               FROM datums
#         # #               WHERE 
#         # #                 (
#         # #                   uri.extra['nuscenes-label-channel'] is NULL OR 
#         # #                   uri.extra['nuscenes-label-channel'] LIKE '%LIDAR%'
#         # #                 ) AND (
#         # #                   uri.topic LIKE '%cuboid%' OR
#         # #                   uri.topic LIKE '%lidar%'
#         # #                 )
#         # #               GROUP BY task_id
#         # #               HAVING SIZE(cuboids) > 0 AND SIZE(point_clouds) > 0
#         # #             """)
        
        
        
#         task_data_df = spark.sql("""
#             SELECT 
#               COLLECT_LIST(STRUCT(__pyclass__, uri, point_cloud)) 
#                   FILTER (WHERE uri.topic LIKE '%lidar%') AS pc_sds,
#               COLLECT_LIST(STRUCT(__pyclass__, uri, cuboids)) 
#                   FILTER (WHERE uri.topic LIKE '%cuboid%') AS cuboids_sds,
#               COLLECT_LIST(STRUCT(__pyclass__, uri, camera_image)) 
#                   FILTER (WHERE uri.topic LIKE '%camera%') AS ci_sds,
#               MIN(uri.timestamp) FILTER (WHERE uri.topic LIKE '%lidar%') AS lidar_time,
#               FIRST(uri.extra.`nuscenes-sample-token`) AS sample_token
#             FROM datums
#             WHERE 
#             uri.extra.`nuscenes-is-keyframe` = 'True' AND (
#               uri.extra['nuscenes-label-channel'] is NULL OR 
#               uri.extra['nuscenes-label-channel'] LIKE '%LIDAR%'
#             ) AND (
#               uri.topic LIKE '%cuboid%' OR
#               uri.topic LIKE '%lidar%' OR
#               uri.topic LIKE '%camera%'
#             )
#             GROUP BY uri.extra.`nuscenes-sample-token`
#             ORDER BY lidar_time
#         """)
#         sample_tokens_ordered = [r.sample_token for r in task_data_df.select('sample_token').collect()]
#         task_to_stoken = [
#             {'task_id': task_id, 'sample_token': sample_token}
#             for task_id, sample_token in enumerate(sample_tokens_ordered)
#         ]
#         task_id_rdd = spark.sparkContext.parallelize(task_to_stoken)
#         task_id_df = spark.createDataFrame(task_id_rdd)
#         tasks_df = task_data_df.join(task_id_df, on=['sample_token'], how='inner')
#         tasks_df = tasks_df.persist()
#         print('... done.')
#         return tasks_df
    
class NuscWorldCloudTableBase(FusedLidarCloudTableBase):
    SPLITS = ['train_detect', 'train_track']
    
    @classmethod
    def _filter_ego_vehicle(cls, cloud_ego):
        # Note: NuScenes authors have already corrected clouds for ego motion:
        # https://github.com/nutonomy/nuscenes-devkit/issues/481#issuecomment-716250423
        # But have not filtered out ego self-returns
        cloud_ego = cloud_ego[np.where(  ~(
                        (cloud_ego[:, 0] <= 1.5) & (cloud_ego[:, 0] >= -1.5) &  # Nusc lidar +x is +right
                        (cloud_ego[:, 1] <= 2.5) & (cloud_ego[:, 0] >= -2.5) &  # Nusc lidar +y is +forward
                        (cloud_ego[:, 1] <= 1.5) & (cloud_ego[:, 0] >= -1.5)    # Nusc lidar +z is +up
        ))]
        return cloud_ego
    
class NuscKFOnlyFusedWorldCloudTable(NuscWorldCloudTableBase):
    TASK_DF_FACTORY = NuscKFOnlyLCCDFFactory

class NuscAllFramesFusedWorldCloudTable(NuscWorldCloudTableBase):
    TASK_DF_FACTORY = NuscAllFramesLCCDFFactory
    
    
#     task_id=int(scan_id),
#                     pc_sds=point_clouds,
#                     cuboids_sds=[], # SemanticKITTI has no cuboids
#                     ci_sds=camera_images
#     @classmethod
#     def _get_task_lidar_cuboid_rdd(cls, spark, segment_uri):
#         datum_df = cls.SRC_SD_TABLE.get_segment_datum_df(spark, segment_uri)
#         datum_df.registerTempTable('datums')
#         spark.catalog.dropTempView('nusc_task_df')
#         print('Building tasks table for %s ...' % segment_uri.segment_id)
        
#         # Nusc doesn't have numerical task_ids so we'll have to induce
#         # one via lidar timestamp.
#         if cls.SRC_SD_TABLE.LABELS_KEYFRAMES_ONLY:
#             # For Nusc: group by nuscenes-sample-token WITH KEYFRAMES
#             spark.sql("""
#               CACHE TABLE nusc_task_df OPTIONS ( 'storageLevel' 'DISK_ONLY' ) AS
#               SELECT 
#                   MIN(uri.timestamp) FILTER (WHERE uri.topic LIKE '%lidar%') AS task_id,
#                   COLLECT_LIST(*) FILTER (WHERE uri.topic LIKE '%lidar%') AS pc_sds,
#                   COLLECT_LIST(*) FILTER (WHERE uri.topic LIKE '%cuboid%') AS cuboids_sds,
#                   COLLECT_LIST(*) FILTER (WHERE uri.topic LIKE '%cam%') AS ci_sds
#               FROM datums
#               WHERE 
#                 uri.extra.`nuscenes-is-keyframe` = 'True' AND (
#                   uri.extra['nuscenes-label-channel'] is NULL OR 
#                   uri.extra['nuscenes-label-channel'] LIKE '%LIDAR%' OR
#                   uri.extra['nuscenes-label-channel'] LIKE '%CAM%'
#                 ) AND (
#                   uri.topic LIKE '%cuboid%' OR
#                   uri.topic LIKE '%lidar%' OR
#                   uri.topic LIKE '%cam%'
#                 )
#               GROUP BY task_id
#             """)
#         else:
#             # For Nusc: group by nuscenes-sample-token WITH ALL FRAMES
#             spark.sql("""
#               CACHE TABLE nusc_task_df OPTIONS ( 'storageLevel' 'DISK_ONLY' ) AS
#               SELECT 
#                   CONCAT(uri.segment_id, '.', uri.timestamp) AS task_id,
#                   FLATTEN(COLLECT_LIST(cuboids)) AS cuboids, 
#                   COLLECT_LIST(point_cloud) AS point_clouds
#               FROM datums
#               WHERE 
#                 (
#                   uri.extra['nuscenes-label-channel'] is NULL OR 
#                   uri.extra['nuscenes-label-channel'] LIKE '%LIDAR%'
#                 ) AND (
#                   uri.topic LIKE '%cuboid%' OR
#                   uri.topic LIKE '%lidar%'
#                 )
#               GROUP BY task_id
#               HAVING SIZE(cuboids) > 0 AND SIZE(point_clouds) > 0
#             """)
# #             spark.sql("""
# #               CACHE TABLE culi_tasks_df OPTIONS ( 'storageLevel' 'DISK_ONLY' ) AS
# #               SELECT 
# #                   CONCAT(uri.segment_id, '.', uri.timestamp) AS task_id,
# #                   FLATTEN(COLLECT_LIST(cuboids)) AS cuboids, 
# #                   COLLECT_LIST(point_cloud) AS point_clouds
# #               FROM datums
# #               WHERE 
# #                 (
# #                   uri.extra['nuscenes-label-channel'] is NULL OR 
# #                   uri.extra['nuscenes-label-channel'] LIKE '%LIDAR%'
# #                 ) AND (
# #                   uri.topic LIKE '%cuboid%' OR
# #                   uri.topic LIKE '%lidar%'
# #                 )
# #               GROUP BY task_id
# #               HAVING SIZE(cuboids) > 0 AND SIZE(point_clouds) > 0
# #             """)
        
#         tasks_df = spark.sql('SELECT * FROM nusc_task_df')
#         print('... done.')
#         return tasks_df.rdd

# class NuscFusedWorldCloudKeyframesOnlyTable(NuscFusedWorldCloudTableBase):
#     SRC_SD_TABLE = NuscStampedDatumTableBase

# class NuscFusedWorldCloudAllFramesTable(NuscFusedWorldCloudTableBase):
#     SRC_SD_TABLE = NuscStampedDatumTableLabelsAllFrames
    
class NuscKeyframesOFlowRenderer(OpticalFlowRenderBase):
    FUSED_LIDAR_SD_TABLE = NuscKFOnlyFusedWorldCloudTable

class NuscAllFramesOFlowRenderer(OpticalFlowRenderBase):
    FUSED_LIDAR_SD_TABLE = NuscAllFramesFusedWorldCloudTable

## Start Spark

In [5]:
from psegs.spark import NBSpark
spark = NBSpark.getOrCreate()

2021-02-22 09:40:58,644	oarph 4020481 : Using source root /opt/psegs/psegs 
2021-02-22 09:40:58,644	oarph 4020481 : Using source root /opt/psegs 
2021-02-22 09:40:58,726	oarph 4020481 : Generating egg to /tmp/tmpt3_mak3v_oarphpy_eggbuild ...
2021-02-22 09:40:58,820	oarph 4020481 : ... done.  Egg at /tmp/tmpt3_mak3v_oarphpy_eggbuild/psegs-0.0.0-py3.8.egg


## Build Fused Lidar Assets

```
docker --context default run -it --name=potree_viewer --rm --net=host -v `pwd`:/shared  jonazpiazu/potree
```

In [6]:
# T = KITTI360OurFusedWorldCloudTable
# rdds = T._create_datum_rdds(spark)
# print([r.count() for r in rdds])

# seg_uris = T.get_all_segment_uris()
# samp = T.get_sample(seg_uris[0], spark=spark)


In [7]:
# print([lc.sensor_name for lc in samp.lidar_clouds][:10])
# c = samp.lidar_clouds[0]#[lc for lc in samp.lidar_clouds if lc.sensor_name == '11002'][0]
# print(c.get_cloud().shape)
# imshow(c.get_bev_debug_image(x_bounds_meters=None, y_bounds_meters=None))
# imshow(c.get_front_rv_debug_image(y_bounds_meters=None, z_bounds_meters=None))

## Compute Candidate Optical Flow Pairs

## Render Optical Flow

In [ ]:
# R = NuscKeyframesOFlowRenderer
# R.MAX_TASKS_PER_SEGMENT = 5
R = SemanticKITTIOFlowRenderer
R.MAX_TASKS_PER_SEGMENT = 5
seg_uris = R.FUSED_LIDAR_SD_TABLE.get_all_segment_uris()
R.build(spark=spark, only_segments=[seg_uris[0]])

2021-02-22 09:41:02,288	ps   4020481 : Found Sequence 00 with 4541 scans
2021-02-22 09:41:02,291	ps   4020481 : Found Sequence 01 with 1101 scans
2021-02-22 09:41:02,303	ps   4020481 : Found Sequence 02 with 4661 scans
2021-02-22 09:41:02,305	ps   4020481 : Found Sequence 03 with 801 scans
2021-02-22 09:41:02,310	ps   4020481 : Found Sequence 05 with 2761 scans
2021-02-22 09:41:02,313	ps   4020481 : Found Sequence 06 with 1101 scans
2021-02-22 09:41:02,316	ps   4020481 : Found Sequence 07 with 1101 scans
2021-02-22 09:41:02,320	ps   4020481 : Found Sequence 09 with 1591 scans
2021-02-22 09:41:02,323	ps   4020481 : Found Sequence 10 with 1201 scans
2021-02-22 09:41:02,323	ps   4020481 : Found 18859 total scans
2021-02-22 09:41:02,325	ps   4020481 : Filtering to only 1 segments
2021-02-22 09:41:02,325	ps   4020481 : Finding scans for sequence 00 with no moving points ...
2021-02-22 09:41:08,703	ps   4020481 : ... sequence 00 has 3097 scans with no movers.
2021-02-22 09:41:23,356	ps   402

num tasks 3097
restrict to 5
oflow_task_df 5
coalesc to  1


In [ ]:
# assert False, spark.sql("""select uri.topic t from datums group by t""").show(truncate=False)

# spark.sql("""
#           CACHE TABLE nusc_task_df OPTIONS ( 'storageLevel' 'DISK_ONLY' ) AS
#           SELECT 
#               MIN(uri.timestamp) FILTER (WHERE uri.topic LIKE '%lidar%') AS task_id,
#               COLLECT_LIST(STRUCT(__pyclass__, uri, point_cloud)) FILTER (WHERE uri.topic LIKE '%lidar%') AS pc_sds,
#               COLLECT_LIST(STRUCT(__pyclass__, uri, cuboids)) FILTER (WHERE uri.topic LIKE '%cuboid%') AS cuboids_sds,
#               COLLECT_LIST(STRUCT(__pyclass__, uri, camera_image)) FILTER (WHERE uri.topic LIKE '%cam%') AS ci_sds
#           FROM datums
#           WHERE 
#             uri.extra.`nuscenes-is-keyframe` = 'True' AND (
#               uri.extra['nuscenes-label-channel'] is NULL OR 
#               uri.extra['nuscenes-label-channel'] LIKE '%LIDAR%'
#             ) AND (
#               uri.topic LIKE '%cuboid%' OR
#               uri.topic LIKE '%lidar%' OR
#               uri.topic LIKE '%camera%'
#             )
#           GROUP BY uri.extra.`nuscenes-sample-token`
#         """).show()
# df = spark.sql("""
#           SELECT 
#               COLLECT_LIST(STRUCT(__pyclass__, uri, point_cloud)) 
#                   FILTER (WHERE uri.topic LIKE '%lidar%') AS pc_sds,
#               COLLECT_LIST(STRUCT(__pyclass__, uri, cuboids)) 
#                   FILTER (WHERE uri.topic LIKE '%cuboid%') AS cuboids_sds,
#               COLLECT_LIST(STRUCT(__pyclass__, uri, camera_image)) 
#                   FILTER (WHERE uri.topic LIKE '%camera%') AS ci_sds,
#               MIN(uri.timestamp) FILTER (WHERE uri.topic LIKE '%lidar%') AS lidar_time,
#               FIRST(uri.extra.`nuscenes-sample-token`) AS sample_token
#           FROM datums
#           WHERE 
#             uri.extra.`nuscenes-is-keyframe` = 'True' AND (
#               uri.extra['nuscenes-label-channel'] is NULL OR 
#               uri.extra['nuscenes-label-channel'] LIKE '%LIDAR%'
#             ) AND (
#               uri.topic LIKE '%cuboid%' OR
#               uri.topic LIKE '%lidar%' OR
#               uri.topic LIKE '%camera%'
#             )
#           GROUP BY uri.extra.`nuscenes-sample-token`
#           ORDER BY lidar_time
#         """)
# sample_tokens_ordered = [r.sample_token for r in df.select('sample_token').collect()]
# task_to_stoken = [
#     {'task_id': task_id, 'sample_token': sample_token}
#     for task_id, sample_token in enumerate(sample_tokens_ordered)
# ]
# task_id_rdd = spark.sparkContext.parallelize(task_to_stoken)
# task_id_df = spark.createDataFrame(task_id_rdd)
# df.join(task_id_df, on=['sample_token'], how='inner').orderBy('task_id').show()